<h1>ТЗ Data Analyst

Выполнила: Кострыгина Елизавета

<h3>
1. Подключение необходимых библиотек

In [20]:
import pandas as pd
import numpy as np
from google.colab import files

<h3> Загрузка датасетов

In [21]:
financial_data = pd.read_csv('financial_data.csv')
prolongations = pd.read_csv('prolongations.csv')

<h3> 2. Разведочный анализ данных

Просмотр данных

In [22]:
financial_data.head()

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович


In [23]:
prolongations.head()

,id,month,AM
0,42,ноябрь 2022,Васильев Артем Александрович
1,453,ноябрь 2022,Васильев Артем Александрович
2,548,ноябрь 2022,Михайлов Андрей Сергеевич
3,87,ноябрь 2022,Соколова Анастасия Викторовна
4,429,ноябрь 2022,Соколова Анастасия Викторовна


In [24]:
financial_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             451 non-null    int64 
 1   Причина дубля  301 non-null    object
 2   Ноябрь 2022    156 non-null    object
 3   Декабрь 2022   159 non-null    object
 4   Январь 2023    139 non-null    object
 5   Февраль 2023   145 non-null    object
 6   Март 2023      168 non-null    object
 7   Апрель 2023    174 non-null    object
 8   Май 2023       190 non-null    object
 9   Июнь 2023      190 non-null    object
 10  Июль 2023      195 non-null    object
 11  Август 2023    199 non-null    object
 12  Сентябрь 2023  186 non-null    object
 13  Октябрь 2023   182 non-null    object
 14  Ноябрь 2023    171 non-null    object
 15  Декабрь 2023   146 non-null    object
 16  Январь 2024    95 non-null     object
 17  Февраль 2024   101 non-null    object
 18  Account        451 non-null   

In [25]:
prolongations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      477 non-null    int64 
 1   month   477 non-null    object
 2   AM      477 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.3+ KB


<h3> 3. Подготовка датаcетов к расчетам

Обработаю financial_data:

Удаляю строки, в которых имеется стоп или end, так как их не нужно учитывать в пролонгации


In [26]:
mask = (financial_data == 'стоп') | (financial_data == 'end')
financial_data = financial_data.loc[~mask.any(axis=1)]

Удаляю столбцы 'Причина дубля' , так как эта информация не несет смысла в нашем случае, и 'Account', так как ФИО менеджеров в столбце 'AM' первичные данные

In [27]:
financial_data = financial_data.drop(columns=['Причина дубля', 'Account'], axis=1)

Преобразую числа в float(id тоже), 'в ноль' и NaN заменяю на '0'

In [28]:
month_and_id = [col for col in financial_data.columns]

def to_float(value):
    if isinstance(value, str):
        # убираю обычные пробелы и неразрывные пробелы
        val = value.strip().lower().replace('\xa0', '').replace(' ', '').replace(',', '.')
        if val in ('вноль', 'в ноль', 'nan', ''):
            return 0.0
        return float(val)
    return 0.0 if pd.isna(value) else float(value)

for col in month_and_id:
    financial_data[col] = financial_data[col].apply(to_float)

Объединяю строки с одинаковым id

In [29]:
month = [col for col in financial_data.columns if col not in ['id']]
agg_col = {col: 'sum' for col in month}

fd_grouped = financial_data.groupby('id', as_index=False).agg(agg_col)
fd_grouped.head()

,id,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024
0,15.0,439280.0,439280.0,102433.75,102433.75,102433.75,138158.0,138158.0,102433.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31.0,55100.0,55100.0,0.00,44775.00,44775.00,44775.0,44775.0,44775.00,44775.0,44775.0,44775.0,44775.0,44775.0,44775.0,44775.0,46200.0
3,39.0,137700.0,137700.0,149206.50,149206.50,149206.50,149206.5,149206.5,149206.50,149206.5,149206.5,149206.5,149206.5,149206.5,149206.5,0.0,0.0
4,42.0,36220.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Очистим prolongations от дублей, оставляю лишь самый последний месяц реализации проекта

In [30]:
# словарь месяцев
month_map = {
    'Январь': 1, 'Февраль': 2, 'Март': 3, 'Апрель': 4,
    'Май': 5, 'Июнь': 6, 'Июль': 7, 'Август': 8,
    'Сентябрь': 9, 'Октябрь': 10, 'Ноябрь': 11, 'Декабрь': 12
}

# преобразовываю 'месяц год' -> datetime
def parse_month_year(val: str):
    if pd.isna(val):
        return pd.NaT
    try:
        month_name, year = val.strip().split()
        month = month_map[month_name.capitalize()]
        return pd.Timestamp(year=int(year), month=month, day=1)
    except:
        return pd.NaT

prolongations['month_dt'] = prolongations['month'].apply(parse_month_year)

# оставляю только максимальный месяц для каждого id
id_max = prolongations.groupby('id')['month_dt'].idxmax()
prolongations_clean = prolongations.loc[id_max].reset_index(drop=True)

Объединяю датафреймы, заменяю названия колонок месяцев

In [31]:
df = pd.merge(fd_grouped, prolongations_clean, on = 'id')

not_month = ['id', 'month', 'AM', 'month_dt']

# переименовываю нужные колонки
df.columns = [
    parse_month_year(col) if col not in not_month else col
    for col in df.columns
]

df.head()

,id,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00,2024-01-01 00:00:00,2024-02-01 00:00:00,month,AM,month_dt
0,15.0,439280.0,439280.0,102433.75,102433.75,102433.75,138158.0,138158.0,102433.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,июнь 2023,Иванова Мария Сергеевна,2023-06-01
1,16.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,декабрь 2022,Иванова Мария Сергеевна,2022-12-01
2,31.0,55100.0,55100.0,0.00,44775.00,44775.00,44775.0,44775.0,44775.00,44775.0,44775.0,44775.0,44775.0,44775.0,44775.0,44775.0,46200.0,декабрь 2022,Васильев Артем Александрович,2022-12-01
3,39.0,137700.0,137700.0,149206.50,149206.50,149206.50,149206.5,149206.5,149206.50,149206.5,149206.5,149206.5,149206.5,149206.5,149206.5,0.0,0.0,декабрь 2023,Попова Екатерина Николаевна,2023-12-01
4,42.0,36220.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ноябрь 2022,Васильев Артем Александрович,2022-11-01


<h3> 4. Рассчеты

Добавляю 2 столбца: какой коэффициент подходит проекту. Если в следующих месяцах после окончания проекта нет отгрузки - оба коэффициента False, в числителе при подсчете они не фигурируют, а в знаменатель входят.

In [32]:
def coefficient_selection(row):
  last_month = row['month_dt']
  next_month = (last_month + pd.DateOffset(months=1)).to_period('M').to_timestamp()
  second_month = (last_month + pd.DateOffset(months=2)).to_period('M').to_timestamp()

  k_1 = False
  k_2 = False

  if next_month in df.columns and row[next_month] != 0:
        k_1 = True
  elif second_month in df.columns and row[second_month] != 0:
        k_2 = True

  return pd.Series({'select_k_1': k_1, 'select_k_2': k_2})

df[['select_k_1', 'select_k_2']] = df.apply(coefficient_selection, axis=1)
df.head()

,id,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,...,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00,2024-01-01 00:00:00,2024-02-01 00:00:00,month,AM,month_dt,select_k_1,select_k_2
0,15.0,439280.0,439280.0,102433.75,102433.75,102433.75,138158.0,138158.0,102433.75,0.0,...,0.0,0.0,0.0,0.0,0.0,июнь 2023,Иванова Мария Сергеевна,2023-06-01,False,False
1,16.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,декабрь 2022,Иванова Мария Сергеевна,2022-12-01,False,False
2,31.0,55100.0,55100.0,0.00,44775.00,44775.00,44775.0,44775.0,44775.00,44775.0,...,44775.0,44775.0,44775.0,44775.0,46200.0,декабрь 2022,Васильев Артем Александрович,2022-12-01,False,True
3,39.0,137700.0,137700.0,149206.50,149206.50,149206.50,149206.5,149206.5,149206.50,149206.5,...,149206.5,149206.5,149206.5,0.0,0.0,декабрь 2023,Попова Екатерина Николаевна,2023-12-01,False,False
4,42.0,36220.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,ноябрь 2022,Васильев Артем Александрович,2022-11-01,False,False


Коэффициент пролонгации отдела за 2023 год:

In [33]:
# создаю дополнительные столбцы, в которых будут лежать нужные для вычисления даты
df['next_month'] = (df['month_dt'] + pd.DateOffset(months=1)).dt.to_period('M').dt.to_timestamp()
df['second_month'] = (df['month_dt'] + pd.DateOffset(months=2)).dt.to_period('M').dt.to_timestamp()

def get_numerator(row):
  if row['select_k_1'] == True:     # если нужно посчитать k_1, возвращаю значение в следующем месяце
    return row[row['next_month']]
  elif row['select_k_2'] == True:   # если нужно посчитать k_2, возвращаю значение через 2 месяца
    return row[row['second_month']]

def get_denominator(row):
  last_month = row['month_dt']
  prior_month = (last_month - pd.DateOffset(months=1)).replace(day=1)

  if last_month in df.columns and row[last_month] != 0: # обрабатываю "в ноль": если нет суммы
    return row[last_month]                              # в последнем месяце, беру сумму за предыдущий
  elif prior_month in df.columns:
    return row[prior_month]

  return 0

df['numerator'] = df.apply(get_numerator, axis=1)
df['denominator'] = df.apply(get_denominator, axis=1)

# группирую и суммирую знаменатель и числитель
grouped = df.groupby(['select_k_1', 'select_k_2'])[['numerator', 'denominator']].sum()

# в числитель коэффициента 1 беру проекты, у которых есть отгрузка в следующем
# после окончания месяце
sum_11 = grouped.loc[(True, False), 'numerator']
# в знаменатель беру все проекты, завершившиеся в прошлом месяце
sum_12 = grouped.loc[(True, False), 'denominator']
sum_12 += grouped.loc[(False, True), 'denominator']
sum_12 += grouped.loc[(False, False), 'denominator']
# в числитель коэффициента 2 беру отгрузку через 2 месяца после окончания
sum_21 = grouped.loc[(False, True), 'numerator']
# в знаменатель только те проекты, что не пролонгированы в первый месяц
sum_22 = grouped.loc[(False, True), 'denominator']
sum_22 += grouped.loc[(False, False), 'denominator']

k_1 = round(sum_11 / sum_12, 3)
k_2 = round(sum_21 / sum_22, 3)

department_coefficients_for_the_year = pd.DataFrame({
    'Год': [2023],
    'k_1': [k_1],
    'k_2': [k_2]
})

department_coefficients_for_the_year.head()

,Год,k_1,k_2
0,2023,0.307,0.051


Коэффициент пролонгации отдела за каждый месяц 2023:



In [34]:
df['numerator'] = df.apply(get_numerator, axis=1)
df['denominator'] = df.apply(get_denominator, axis=1)
results = []

for month in pd.period_range('2023-01', '2023-12', freq='M'):
  month_ts = month.to_timestamp()

  # отделяю проекты, которые в текущем месяце нужны для подсчета числителя k_1
  group_k_1 = df[df['next_month'] == month_ts]

  # считаю суммы для k_1
  sum_11 = group_k_1.loc[group_k_1['select_k_1'], 'numerator'].sum()
  sum_12 = group_k_1['denominator'].sum()

  # отделяю проекты, которые в текущем месяце нужны для подсчета коэффициента k_2
  group_k_2 = df[df['second_month'] == month_ts]

  # считаю суммы для k_2
  sum_21 = group_k_2.loc[group_k_2['select_k_2'], 'numerator'].sum()
  sum_22 = group_k_2.loc[~group_k_2['select_k_1'],'denominator'].sum()

  k_1 = round(sum_11 / sum_12, 3) if sum_12 != 0 else None
  k_2 = round(sum_21 / sum_22, 3) if sum_22 != 0 else None

  results.append({
        'Месяц': month.to_timestamp(),
        'k_1': k_1,
        'k_2': k_2
    })

department_coefficients_by_months = pd.DataFrame(results)

print(department_coefficients_by_months)

        Месяц    k_1    k_2
0  2023-01-01  0.128  0.000
1  2023-02-01  0.624  0.042
2  2023-03-01  0.000  0.119
3  2023-04-01  0.000  0.000
4  2023-05-01  0.000  0.000
5  2023-06-01  0.000  0.000
6  2023-07-01  0.061  0.079
7  2023-08-01  0.251  0.000
8  2023-09-01  0.140  0.000
9  2023-10-01  0.566  0.000
10 2023-11-01  0.279  0.019
11 2023-12-01  0.491  0.213


Коэффициент пролонгации каждого менеджера за год:

In [35]:
results = []

for am, group in df.groupby(['AM']):
  sum_11 = group.loc[group['select_k_1'], 'numerator'].sum()
  sum_12 = group['denominator'].sum()

  sum_21 = group.loc[group['select_k_2'], 'numerator'].sum()
  sum_22 = group.loc[~group['select_k_1'],'denominator'].sum()

  k_1 = round(sum_11 / sum_12, 3) if sum_12 != 0 else None
  k_2 = round(sum_21 / sum_22, 3) if sum_22 != 0 else None

  results.append({
        'ФИО': am[0],
        'k_1': k_1,
        'k_2': k_2
    })

AM_coefficients_for_year = pd.DataFrame(results)
print(AM_coefficients_for_year)

                             ФИО    k_1    k_2
0   Васильев Артем Александрович  0.236  0.052
1        Иванова Мария Сергеевна  0.143  0.000
2       Кузнецов Михаил Иванович  0.491  0.000
3      Михайлов Андрей Сергеевич  0.386  0.110
4        Петрова Анна Дмитриевна  1.111    NaN
5    Попова Екатерина Николаевна  0.293  0.027
6    Смирнова Ольга Владимировна  0.514  0.199
7  Соколова Анастасия Викторовна  0.292  0.015
8     Федорова Марина Васильевна  0.000  0.000
9                        без А/М  0.000  0.000


Коэффициент пролонгации для каждого менеджера за каждый месяц:

In [36]:
result = []

for month in pd.period_range('2023-01', '2023-12', freq='M'):
  month_ts = month.to_timestamp()

  month_data = df[(df['next_month'] == month_ts) | (df['second_month'] == month_ts)]

  for manager, manager_data in month_data.groupby('AM'):
    group_k_1 = manager_data[manager_data['next_month'] == month_ts]

    sum_11 = group_k_1.loc[group_k_1['select_k_1'], 'numerator'].sum()
    sum_12 = group_k_1['denominator'].sum()

    group_k_2 = manager_data[manager_data['second_month'] == month_ts]

    sum_21 = group_k_2.loc[group_k_2['select_k_2'], 'numerator'].sum()
    sum_22 = group_k_2.loc[~group_k_2['select_k_1'],'denominator'].sum()

    k_1 = round(sum_11 / sum_12, 3) if sum_12 != 0 else None
    k_2 = round(sum_21 / sum_22, 3) if sum_22 != 0 else None

    result.append({
            'Менеджер': manager,
            'Месяц': month.to_timestamp(),
            'k_1': k_1,
            'k_2': k_2
        })
result_df = pd.DataFrame(result)

Для читабельности переименую столбцы с датами обратно:

In [37]:
AM_coefficients_by_months = result_df.pivot_table(
    index = 'Менеджер',
    columns = 'Месяц',
    values = ['k_1', 'k_2'],
    dropna=False
)

AM_coefficients_by_months = AM_coefficients_by_months.swaplevel(0, 1, axis=1).sort_index(axis=1)

AM_coefficients_by_months.head()

Месяц                        2023-01-01      2023-02-01        2023-03-01  \
                                    k_1  k_2        k_1    k_2        k_1   
Менеджер                                                                    
Васильев Артем Александрович       0.00  0.0        0.0  0.205        0.0   
Иванова Мария Сергеевна            0.16  NaN        NaN  0.000        0.0   
Кузнецов Михаил Иванович            NaN  NaN        NaN    NaN        NaN   
Михайлов Андрей Сергеевич          0.00  0.0        1.0  0.000        NaN   
Петрова Анна Дмитриевна             NaN  NaN        NaN    NaN        NaN   

Месяц                             2023-04-01      2023-05-01       ...  \
                              k_2        k_1  k_2        k_1  k_2  ...   
Менеджер                                                           ...   
Васильев Артем Александрович  0.0        0.0  0.0        0.0  0.0  ...   
Иванова Мария Сергеевна       NaN        0.0  0.0        NaN  0.0  ...   
Кузнецов Михаил Иванович      NaN        0.0  NaN        NaN  0.0  ...   
Михайлов Андрей Сергеевич     NaN        NaN  NaN        NaN  NaN  ...   
Петрова Анна Дмитриевна       NaN        NaN  NaN        NaN  NaN  ...   

Месяц                        2023-08-01      2023-09-01      2023-10-01       \
                                    k_1  k_2        k_1  k_2        k_1  k_2   
Менеджер                                                                       
Васильев Артем Александрович      0.297  0.0        0.0  0.0        0.0  0.0   
Иванова Мария Сергеевна           0.380  0.0        1.0  0.0        NaN  NaN   
Кузнецов Михаил Иванович            NaN  NaN        NaN  NaN        NaN  NaN   
Михайлов Андрей Сергеевич           NaN  NaN        0.0  NaN        0.0  0.0   
Петрова Анна Дмитриевна             NaN  NaN        NaN  NaN        NaN  NaN   

Месяц                        2023-11-01      2023-12-01         
                                    k_1  k_2        k_1    k_2  
Менеджер                                                        
Васильев Артем Александрович      0.000  0.0      0.324  0.485  
Иванова Мария Сергеевна             NaN  NaN        NaN    NaN  
Кузнецов Михаил Иванович          1.028  NaN      0.274    NaN  
Михайлов Андрей Сергеевич           NaN  0.0      0.000    NaN  
Петрова Анна Дмитриевна             NaN  NaN      1.111    NaN  

[5 rows x 24 columns]

Создаю excel файл, помещаю в него результаты:

In [38]:
with pd.ExcelWriter('Коэффициенты пролонгации за 2023 год.xlsx', engine='openpyxl') as writer:
    department_coefficients_for_the_year.to_excel(writer, sheet_name='КП отдела за год', index=False)
    department_coefficients_by_months.to_excel(writer, sheet_name='КП отдела за каждый месяц', index=False)
    AM_coefficients_for_year.to_excel(writer, sheet_name='КП менеджеров за год', index=False)
    AM_coefficients_by_months.to_excel(writer, sheet_name='КП менеджеров за каждый месяц', index_label=['k_1', 'k_2'])